# Adaptive Evaluations with Scorebook - Evaluating a Local Qwen Model

This quick-start guide showcases an adaptive evaluation of Qwen's Qwen2.5 0.5B Instruct model.

We recommend that you first see our [getting started quick-start guide](https://colab.research.google.com/github/trismik/scorebook/blob/main/tutorials/quickstarts/getting_started.ipynb) if you have not done so already, for more of a detailed overview on adaptive testing and setting up Trismik credentials.

## Prerequisites

- **Trismik API key**: Generate a Trismik API key from the [Trismik dashboard's settings page](https://app.trismik.com/settings).
- **Trismik Project Id**: We recommend you use the project id generated in the [Getting Started Quick-Start Guide](https://colab.research.google.com/github/trismik/scorebook/blob/main/tutorials/quickstarts/getting_started.ipynb).

## Install Scorebook

In [1]:
!pip install scorebook

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
  A

## Setup Credentials

Enter your Trismik API key and project id below.

In [2]:
# Set your credentials here
TRISMIK_API_KEY = "trismik_XnWx8vpP7GDevP1C1dq7Gu2EcmH5BRVl"
TRISMIK_PROJECT_ID = "359b479e41b567dc279425b8d7cb6411e00f9602"

## Login with Trismik API Key

In [3]:
from scorebook import login

# Login to Trismik
login(TRISMIK_API_KEY)
print("✓ Logged in to Trismik")

✓ Logged in to Trismik


## Instantiate a Local Qwen Model

For this quick-start guide, we will use the lightweight Qwen2.5 0.5B instruct model, via Hugging Face's transformers package.

In [ ]:
import transformers

# Instantiate a model
pretrained = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

pipeline = transformers.pipeline(
    "text-generation",
    model=pretrained,
    tokenizer=pretrained,             # 可加可不加，加上更明确
    model_kwargs={"torch_dtype": "auto"},
    device_map="auto",
)
print("✓ Transformers pipeline instantiated")

## Define an Inference Function

To evaluate a model with Scorebook, it must be encapsulated within an inference function. An inference function must accept a list of model inputs, pass these to the model for inference, collect and return outputs generated.

An inference function can be defined to encapsulate any model, local or cloud-hosted. There is flexibility in how an inference function can be defined, the only requirements are the function signature. An inference function must,

Accept:

- A list of model inputs.
- Hyperparameters which can be optionally accessed via kwargs.

Return

- A list of parsed model outputs for scoring.

In [ ]:
from typing import Any, List
import string

# Define an inference function for the Qwen model.
def qwen(inputs: List[Any], **hyperparameters: Any) -> List[Any]:
    """Process inputs through Qwen model"""

    outputs = []
    for idx, input_item in enumerate(inputs):

        # Format prompt
        choices = input_item.get("options", [])
        prompt = (
            str(input_item.get("question", ""))
            + "\nOptions:\n"
            + "\n".join(
                f"{letter}: {choice['text'] if isinstance(choice, dict) else choice}"
                for letter, choice in zip(string.ascii_uppercase, choices)
            )
        )

        # Build messages for Qwen model
        messages = [
            {
                "role": "system",
                "content": hyperparameters["system_message"]
            },
            {"role": "user", "content": prompt},
        ]

        # Run inference using the pipeline
        try:
            output = pipeline(
                messages,
                temperature = hyperparameters.get("temperature", 0.7),
                top_p = hyperparameters.get("top_p", 0.9),
                top_k = hyperparameters.get("top_k", 50),
                max_new_tokens = 512,
                do_sample = hyperparameters.get("temperature", 0.7) > 0,
            )
            response = output[0]["generated_text"][-1]["content"].strip()

        except Exception as e:
            response = f"Error: {str(e)}"

        outputs.append(response)

    return outputs

## Run an Adaptive Evaluation

When running an adaptive evaluation, we can use any single or multiple adaptive datasets and specify a split to be evaluated.

In [ ]:
from scorebook import evaluate

# Run adaptive evaluation
results = evaluate(
    inference = qwen,
    datasets = "trismik/FinRAG:adaptive",
    hyperparameters = {"system_message": "Answer the question with only the letter of the correct option. No additional text or context"},
    split = "validation",
    experiment_id = "Qwen-2.5-0.5B-Adaptive-Evaluation",
    project_id = TRISMIK_PROJECT_ID,
)

# Print the adaptive evaluation results
print("✓ Adaptive evaluation complete!")
print("Results: ", results[0]["score"])

---

## Next Steps

- [Adaptive Testing White Paper](https://docs.trismik.com/adaptiveTesting/adaptive-testing-introduction/): An in depth overview of the science behind the adaptive testing methodology.
- [Dataset Page](https://dashboard.trismik.com/datasets): Trismik's full set of currently adaptive datasets from the Trismik dashboard.
- [Scorebook Docs](https://docs.trismik.com/scorebook/introduction-to-scorebook/): Scorebook's full documentation.
- [Scorebook Repository](https://github.com/trismik/scorebook): Scorebook is an open-source library, view the code and more examples.